In [1]:
import numpy as np
import csv
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
from  matplotlib import pyplot as plt
from sklearn import cross_validation
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn import metrics, preprocessing
import pickle
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [2]:
df = pd.read_json("~/Documents/AML/HW2/train.json")

In [3]:
df.tail(5)

,cuisine,id,ingredients
39769,irish,29109,"[light brown sugar, granulated sugar, butter, ..."
39770,italian,11462,"[KRAFT Zesty Italian Dressing, purple onion, b..."
39771,irish,2238,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli..."
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground..."


In [4]:
numCuisineInst = df.groupby(['cuisine']).agg(['count'])

In [5]:
numCuisineInst

,id,ingredients
,count,count
cuisine,,
brazilian,467,467
british,804,804
cajun_creole,1546,1546
chinese,2673,2673
filipino,755,755
french,2646,2646
greek,1175,1175
indian,3003,3003


In [14]:
# df.ingredients.nunique()
# Gives error because the list is not hashable
# ways to solve it, loop through every single item as a matrix, and create a dictionary, or create a hash map, or 

In [6]:
ingredients_list = df['ingredients'].as_matrix()

In [7]:
ingredients_list[0:2]

array([ ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles'],
       ['plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']], dtype=object)

In [8]:
len(ingredients_list)

39774

In [9]:
ingredientDict = dict()

In [10]:
for item in ingredients_list:
    for ingredient in item:
        if ingredient in ingredientDict:
            ingredientDict[ingredient] +=1
        else:
            ingredientDict[ingredient] =1

In [11]:
len(ingredientDict)

6714

In [12]:
consolidated_list = list(ingredientDict.keys())

In [13]:
print (consolidated_list[0:2])

['canned tuna', 'turkey breakfast sausage']


In [14]:
blank_data = np.empty((len(ingredients_list), len(consolidated_list)))

In [15]:
blank_data[0:2]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [16]:
df_one_hot_encode = pd.DataFrame(blank_data , columns=consolidated_list)

In [17]:
# df_one_hot_encode = df_one_hot_encode.fillna(0)
df_one_hot_encode.head(5)

,canned tuna,turkey breakfast sausage,ice,liquorice,boneless beef roast,yellow mustard seeds,low-fat crème fraîche,fresh thyme,popcorn,carnitas,...,vodka sauce,gobo root,chipped beef,cut up chicken,medjool date,napa cabbage leaves,curry powder,paneer,reduced sodium chicken stock,habanero hot sauce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
i = 0
j = 0
while i < len(ingredients_list):
    while j < len(ingredients_list[i]):
        df_one_hot_encode.set_value(i, ingredients_list[i][j], 1)
        print("i= " , i, "; j= ",  j, "; ", ingredients_list[i][j])
        j+=1
    i+=1

i=  0 ; j=  0 ;  romaine lettuce
i=  0 ; j=  1 ;  black olives
i=  0 ; j=  2 ;  grape tomatoes
i=  0 ; j=  3 ;  garlic
i=  0 ; j=  4 ;  pepper
i=  0 ; j=  5 ;  purple onion
i=  0 ; j=  6 ;  seasoning
i=  0 ; j=  7 ;  garbanzo beans
i=  0 ; j=  8 ;  feta cheese crumbles
i=  1 ; j=  9 ;  milk
i=  1 ; j=  10 ;  vegetable oil
i=  2 ; j=  11 ;  chicken livers
i=  4 ; j=  12 ;  passata
i=  4 ; j=  13 ;  oil
i=  4 ; j=  14 ;  ground cumin
i=  4 ; j=  15 ;  boneless chicken skinless thigh
i=  4 ; j=  16 ;  garam masala
i=  4 ; j=  17 ;  double cream
i=  4 ; j=  18 ;  natural yogurt
i=  4 ; j=  19 ;  bay leaf
i=  55 ; j=  20 ;  all-purpose flour
i=  55 ; j=  21 ;  ground beef
i=  251 ; j=  22 ;  tomato ketchup
i=  251 ; j=  23 ;  coriander
i=  294 ; j=  24 ;  garlic
i=  294 ; j=  25 ;  filet
i=  294 ; j=  26 ;  thyme leaves
i=  294 ; j=  27 ;  chipotles in adobo
i=  294 ; j=  28 ;  unsweetened cocoa powder
i=  294 ; j=  29 ;  black pepper
i=  294 ; j=  30 ;  honey
i=  294 ; j=  31 ;  boneless s

In [19]:
len(ingredients_list[0])

9

In [20]:
ingredients_list[0][1]

'black olives'

In [114]:
# df_one_hot_encode.set_value(3, 'pecan halves', 0)

In [21]:
df_one_hot_encode.head(5)

,canned tuna,turkey breakfast sausage,ice,liquorice,boneless beef roast,yellow mustard seeds,low-fat crème fraîche,fresh thyme,popcorn,carnitas,...,vodka sauce,gobo root,chipped beef,cut up chicken,medjool date,napa cabbage leaves,curry powder,paneer,reduced sodium chicken stock,habanero hot sauce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df = df.drop( ['ingredients', 'id'],axis=1)

In [23]:
df_full = pd.concat((df, df_one_hot_encode),axis=1)

In [24]:
df_full.head(5)

,cuisine,canned tuna,turkey breakfast sausage,ice,liquorice,boneless beef roast,yellow mustard seeds,low-fat crème fraîche,fresh thyme,popcorn,...,vodka sauce,gobo root,chipped beef,cut up chicken,medjool date,napa cabbage leaves,curry powder,paneer,reduced sodium chicken stock,habanero hot sauce
0,greek,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,southern_us,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,filipino,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,indian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,indian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df_full['cuisine'] = pd.factorize(df_full['cuisine'])[0]

In [26]:
df_full.tail(5)

,cuisine,canned tuna,turkey breakfast sausage,ice,liquorice,boneless beef roast,yellow mustard seeds,low-fat crème fraîche,fresh thyme,popcorn,...,vodka sauce,gobo root,chipped beef,cut up chicken,medjool date,napa cabbage leaves,curry powder,paneer,reduced sodium chicken stock,habanero hot sauce
39769,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39770,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39771,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39772,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39773,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df_full = df_full.dropna()

In [28]:
df_full.tail(5)

,cuisine,canned tuna,turkey breakfast sausage,ice,liquorice,boneless beef roast,yellow mustard seeds,low-fat crème fraîche,fresh thyme,popcorn,...,vodka sauce,gobo root,chipped beef,cut up chicken,medjool date,napa cabbage leaves,curry powder,paneer,reduced sodium chicken stock,habanero hot sauce
39769,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39770,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39771,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39772,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39773,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
Y = df_full['cuisine'].as_matrix()

In [30]:
X =df_full.iloc[:,1:].values

In [31]:
print ("X shape is: ", X.shape, "Y shape is" , Y.shape, )

X shape is:  (39774, 6714) Y shape is (39774,)


In [32]:
##make sure there are no nan values
np.any(np.isnan(X)), np.any(np.isnan(Y))

(False, False)

In [35]:
if 1 in X:
    print(True)

True


In [36]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
    X, Y, test_size=0.25, random_state=42)

In [37]:
clf = LogisticRegression(C=1, multi_class= 'ovr')
clf.fit((X_train),Y_train)
clf.score((X_test),Y_test)

0.19609814963797265

In [38]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, Y_train).predict(X_test)

In [39]:
print("Number of mislabeled points out of a total %d points : %d", (X_test[0],(Y_test != y_pred).sum()))

Number of mislabeled points out of a total %d points : %d (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 8882)


In [40]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred_bnb = bnb.fit(X_train, Y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d", (X_test[0],((Y_test != y_pred).sum())/len(Y_test)))

Number of mislabeled points out of a total %d points : %d (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 0.89320193081255028)


In [42]:
df_full.head(5)

,cuisine,canned tuna,turkey breakfast sausage,ice,liquorice,boneless beef roast,yellow mustard seeds,low-fat crème fraîche,fresh thyme,popcorn,...,vodka sauce,gobo root,chipped beef,cut up chicken,medjool date,napa cabbage leaves,curry powder,paneer,reduced sodium chicken stock,habanero hot sauce
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
